# Generate Passport Index datasets
* Data by Passport Index 2024: https://www.passportindex.org/
* In both tidy and matrix formats
* Using ISO-2, ISO-3, and full country names

In [1]:
import pandas as pd
import json
import time

# Load tqdm!
from tqdm import tqdm
tqdm.pandas()

### In your browser,

1. Navigate to https://www.passportindex.org/comparebyPassport.php?p1=sa&y1=2024
1. Open Dev tools > Network
1. Force-reload the page (e.g. Cmd+Shift+R)
1. Find a POST request to *compare2.php*, right-click, and select 'Copy Value' -> 'Copy as cURL'
1. Paste below

In [2]:
curl_command = """
curl 'https://www.passportindex.org/incl/compare2.php' --compressed -X POST -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:132.0) Gecko/20100101 Firefox/132.0' -H 'Accept: */*' -H 'Accept-Language: en-GB,en;q=0.5' -H 'Accept-Encoding: gzip, deflate, br, zstd' -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' -H 'X-Requested-With: XMLHttpRequest' -H 'Origin: https://www.passportindex.org' -H 'DNT: 1' -H 'Sec-GPC: 1' -H 'Alt-Used: www.passportindex.org' -H 'Connection: keep-alive' -H 'Referer: https://www.passportindex.org/comparebyPassport.php?p1=sa&y1=2024' -H 'Cookie: _ga_7Z48X951ET=GS1.1.1731362927.4.0.1731362932.0.0.0; _ga=GA1.1.982642551.1730071472; PHPSESSID=0833ebcdbd2b18a09ae244ed8b0328e8; cf_clearance=OTj93CXI79Dqe1kSamKHTFA.JH76B6ieNVRU8XPqep0-1731362927-1.2.1.1-P05yJ9K9vSiJrNbRHjQ3ZQaw_rX515yhbDgM3EXOqbYeKHOuVoAzv75RzrN.zKeccTACSiFrwvPirZYSi04cu7PssRtsA7_lSG9jfwFr7TijmN47NFnHujT49zbiNpRTxXEyh7kM7Fc7F5Oqf.Xe8TUKW37mqgdQLQ9XeH2fitHpk6.yIKF9UOHKFYrSAptfZ6XvlcP_4pT39IcJSrndwxKO_sqKHujE0yLCyg.SJtYVqa24J_oCUohIxE3YvPMlhuRrutnrCYfUc18uA34Orrt0skG7vNWotkT5.ypnqmRVgBV7A54lMefYty53xkdfEvBWC3hp4u4ykT0hJV4OFlFFL87rBZm4O6mN1rGuSyg' -H 'Sec-Fetch-Dest: empty' -H 'Sec-Fetch-Mode: cors' -H 'Sec-Fetch-Site: same-origin' -H 'TE: trailers' --data-raw 'compare=3&year=2024&cc=sa&csrf_token=3d9d37a001347533f8beca32b6764b458a17fe33ab7a568da7d5caf02b4df3a6'
"""

In [3]:
# Read all country codes

country_data = (
    pd.read_csv(
        'https://gist.githubusercontent.com/ilyankou/b2580c632bdea4af2309dcaa69860013/raw/420fb417bcd17d833156efdf64ce8a1c3ceb2691/country-codes',
        dtype=str
    )
    .fillna('NA')
)

In [4]:
def get_data(country_code):

    time.sleep(6.1)

    curl_command_ = (curl_command
                     .replace('curl ', 'curl -s ')
                     .replace(', br, zstd', '')
                     .replace('cc=sa', f'cc={str.lower(country_code)}')
                    )

    res = !{curl_command_}
    return json.loads(res[0])

In [5]:
country_data['travel'] = country_data.ISO2.progress_apply(get_data)

100%|█████████████████████████████████████████| 199/199 [21:43<00:00,  6.55s/it]


## Get data from PassportIndex

In [6]:
assert country_data['travel'].isna().sum() == 0, \
    f"Didn't fetch {country_data['travel'].isna().sum()} rows!"

## Clean up the data

In [7]:
obj = {}

for passport in country_data.ISO2.tolist():
    
    # Add passport to the object
    if passport not in obj:
        obj[passport] = {}
    
    # Add destinations for the given passport
    for dest in country_data.query(f'ISO2 == "{passport}"').iloc[0]['travel']:
        
        text = dest['text'].lower()
        res = ''
        
        # ** Visa required, incl Cuba's tourist card or China's Exit Entry Permit for Macau/HK **
        if text == 'visa required' or text == 'tourist card' or text == 'exit-entry permit':
            res = 'visa required'
        
        # ** Visa on arrival **
        elif 'visa on arrival' in text:
            res = 'visa on arrival'
            
        # ** Covid-19 ban ** 
        elif text == 'covid-19 ban':
            res = 'covid ban'
            
        # ** Visa-free, incl. Seychelles' tourist registration **
        elif 'visa-free' in text or 'tourist registration' in text or 'visa waiver' in text:
            res = dest['dur'] if dest['dur'] != '' else 'visa free'
            
        # ** eVisas, incl eVisitors (Australia), eTourist cards (Suriname),
        # eTA (US), and pre-enrollment (Ivory Coast), or EVW (UK) **
        elif 'evis' in text or 'etourist' in text or text == 'eta' or text == 'pre-enrollment' or text == 'evw':
            res = 'e-visa'
            
        # ** No admission, including Trump ban **
        elif text == 'trump ban' or text == 'not admitted':
            res = 'no admission'

        elif 'arrival card' in text or 'e-ticket' in text:
            res = 'visa free'
        
        # Update the result!
        obj[passport][ dest['code'] ] = res if res != '' else dest['text']

In [8]:
for x, y in country_data.head(5).iterrows():
    print(x, y['ISO3'])

0 AFG
1 ALB
2 DZA
3 AND
4 AGO


## Save

In [9]:
# ISO-2: Matrix
matrix = pd.DataFrame(obj).T.fillna(-1)
matrix.to_csv('passport-index-matrix-iso2.csv', index_label='Passport')

# ISO-2: Tidy
matrix.stack().to_csv(
    'passport-index-tidy-iso2.csv',
    index_label=['Passport', 'Destination'],
    header=['Requirement'])

# ISO-3: Matrix
iso2to3 =  { y['ISO2']:y['ISO3'] for _,y in country_data.iterrows() }
matrix.rename(columns=iso2to3, index=iso2to3).to_csv('passport-index-matrix-iso3.csv', index_label='Passport')

# ISO-3: Tidy
matrix.rename(columns=iso2to3, index=iso2to3).stack().to_csv(
    'passport-index-tidy-iso3.csv',
    index_label=['Passport', 'Destination'],
    header=['Requirement'])


# Country names: Matrix
iso2name =  { y['ISO2']:y['Country'] for _,y in country_data.iterrows() }
matrix.rename(columns=iso2name, index=iso2name).to_csv('passport-index-matrix.csv', index_label='Passport')

# Country names: Tidy
matrix.rename(columns=iso2name, index=iso2name).stack().to_csv(
    'passport-index-tidy.csv',
    index_label=['Passport', 'Destination'],
    header=['Requirement'])

In [10]:
# Print all values
tidy = matrix.rename(columns=iso2to3, index=iso2to3).stack()
tidy.value_counts()

visa required      13178
90                  7648
e-visa              7374
visa on arrival     5911
visa free           1883
30                  1659
180                  734
120                  233
-1                   199
60                   192
15                   122
14                   106
21                   106
360                   97
42                    60
no admission          34
28                    23
240                   15
45                    15
10                     6
7                      4
150                    1
31                     1
Name: count, dtype: int64

In [11]:
#tidy[tidy == "Exit-entry Permit"]

In [12]:
# Which countries have no admission? (Typically war zones etc)
tidy[ tidy == 'no admission' ]

ARM  AZE    no admission
BGD  IRQ    no admission
     LBY    no admission
HTI  SUR    no admission
IRN  LBY    no admission
ISR  DZA    no admission
     BGD    no admission
     BRN    no admission
     IRN    no admission
     LBN    no admission
     LBY    no admission
     MYS    no admission
     PAK    no admission
     SAU    no admission
     SYR    no admission
     YEM    no admission
XKX  ARM    no admission
     KHM    no admission
     CUB    no admission
     HKG    no admission
     SYC    no admission
PRK  JPN    no admission
     LKA    no admission
PAK  LBY    no admission
PSE  MDG    no admission
     SYR    no admission
PHL  KWT    no admission
SOM  AUS    no admission
     CAN    no admission
SDN  LBY    no admission
SYR  LBY    no admission
TWN  GEO    no admission
TJK  KGZ    no admission
YEM  LBY    no admission
dtype: object

In [13]:
assert tidy.loc[('BLR', 'RUS')] == '90', "Check data!"

### Difference with previous run
* Typically the difference between two consecutive months would be in low hundreds.
* If difference is higher, double-check the data!

In [14]:
x = (tidy
 .reset_index()
 .rename(
     columns={'level_0': 'Passport', 'level_1': 'Destination', 0: 'Requirement'}
 )
 .merge(
     pd.read_csv('legacy/2024-10-28/passport-index-tidy-iso3.csv'), # SELECT VERSION TO COMPARE!
     how='left',
     left_on=['Passport', 'Destination'],
     right_on=['Passport', 'Destination'],
     suffixes=('_new', '_old')
 )
 .assign(
     is_different=lambda df_: df_.Requirement_old.ne(df_.Requirement_new)
 )
 .query('is_different & (Passport != Destination)')
 .drop(columns=['is_different'])
)

x

,Passport,Destination,Requirement_new,Requirement_old
631,AND,CHN,15,visa required
683,AND,KAZ,90,e-visa
1509,ARM,MNE,visa required,30
9188,DNK,CHN,15,visa required
11775,FIN,CHN,15,visa required
15158,ISL,CHN,15,visa required
17414,KAZ,MAC,14,visa on arrival
17429,KAZ,MNE,visa required,30
19587,LBY,JOR,visa on arrival,visa required
19735,LIE,CHN,15,visa required


In [15]:
# What changed for the UK?
x.query('Passport == "GBR"')

,Passport,Destination,Requirement_new,Requirement_old


In [16]:
x.query('Destination == "GBR"')

,Passport,Destination,Requirement_new,Requirement_old
